In [1]:
import pickle
import os
import numpy as np

project_folder = "."

with open(os.path.join(project_folder,"data","train_test.pkl"), "rb") as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

In [2]:
type(X_test)

scipy.sparse.coo.coo_matrix

In [3]:
X_train_np = X_train.toarray()
X_test_np = (X_test.toarray()) 


y_train_np = np.array(y_train)
y_test_np  = np.array(y_test)

In [4]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
np_data_scaled = sc.fit_transform(X_train_np)
np_data_scaled_test = sc.transform(X_test_np)

In [5]:
### Imports 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.utils import normalize, to_categorical
from sklearn.metrics import roc_curve, auc
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers
from tensorflow.keras.layers import LeakyReLU

In [6]:
model = Sequential()

model.add(tf.keras.Input(shape=(68,)))

model.add(Dense(500))
model.add(LeakyReLU())
model.add(Dropout(rate=0.3))

model.add(Dense(300))
model.add(LeakyReLU())
model.add(Dropout(rate=0.2))

model.add(Dense(100))
model.add(LeakyReLU())

model.add(Dense(1,activation = 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               34500     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 500)               0         
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               150300    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               3

In [7]:
# Compiling the ANN

earlyStopping = EarlyStopping(monitor='val_auc', mode='max', min_delta = 0.0001 , patience = 10,restore_best_weights=True)

callbacks_a = [earlyStopping]

adam = optimizers.Adam(lr = 0.0001)

model.compile(optimizer = optimizers.Adam(), loss = 'binary_crossentropy', metrics = [tf.keras.metrics.AUC(),tf.keras.metrics.Recall()])

In [8]:
# Fitting the ANN
model.fit(np_data_scaled, y_train_np, batch_size = 256, validation_data=(np_data_scaled_test,y_test_np), epochs = 1000 , callbacks = callbacks_a)

Train on 128000 samples, validate on 32000 samples
Epoch 1/1000
128000/128000 [==============================] - 9s 73us/sample - loss: 0.3456 - auc: 0.9239 - recall: 0.7838 - val_loss: 0.2466 - val_auc: 0.9617 - val_recall: 0.8773
Epoch 2/1000
128000/128000 [==============================] - 9s 70us/sample - loss: 0.2372 - auc: 0.9646 - recall: 0.8730 - val_loss: 0.1905 - val_auc: 0.9762 - val_recall: 0.9039
Epoch 3/1000
128000/128000 [==============================] - 9s 70us/sample - loss: 0.2030 - auc: 0.9738 - recall: 0.8967 - val_loss: 0.1906 - val_auc: 0.9764 - val_recall: 0.9145
Epoch 4/1000
128000/128000 [==============================] - 8s 65us/sample - loss: 0.1863 - auc: 0.9778 - recall: 0.9091 - val_loss: 0.1679 - val_auc: 0.9812 - val_recall: 0.9230
Epoch 5/1000
128000/128000 [==============================] - 8s 65us/sample - loss: 0.1733 - auc: 0.9806 - recall: 0.9149 - val_loss: 0.1507 - val_auc: 0.9845 - val_recall: 0.9305
Epoch 6/1000
128000/128000 [================

In [9]:
from sklearn import metrics
import numpy as np
from tqdm import tqdm

probas = list(np.arange(0.01, 0.25, 0.01))

def calcCost(model, X, y, proba, fp = 10, fn = 500):
    y_hat_proba = model.predict(X,use_multiprocessing=True)
    y_hat = (y_hat_proba > proba).astype(int)
    mt = metrics.confusion_matrix(y, y_hat)
    acc = metrics.accuracy_score(y, y_hat)
    precision = metrics.precision_score(y, y_hat)
    return {
        "probability":proba,
        "accuracy": acc,
        "precision": precision,
        "fn":mt[1,0],
        "fn_cost":mt[1,0]*fn,
        "fp":mt[0,1],
        "fp_cost":mt[0,1]*fp
    }

cost = [ calcCost(model, np_data_scaled_test, y_test, proba) for proba in tqdm(probas) ]

100%|██████████| 24/24 [00:35<00:00,  1.47s/it]


In [10]:
import pandas as pd

cost_df = pd.DataFrame(cost)

cost_df["total_cost"] = cost_df.fn_cost + cost_df.fp_cost

cost_df.sort_values(by=["total_cost"], ascending=True).head()

,probability,accuracy,precision,fn,fn_cost,fp,fp_cost,total_cost
1,0.02,0.860094,0.742894,90,45000,4387,43870,88870
2,0.03,0.883906,0.778235,108,54000,3607,36070,90070
3,0.04,0.898062,0.800950,119,59500,3143,31430,90930
4,0.05,0.908000,0.817905,131,65500,2813,28130,93630
0,0.01,0.815250,0.684903,72,36000,5840,58400,94400


In [11]:
model.save('models/model_85.h5')